In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x2dfb570
Welcome to JupyROOT 6.30/06


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetPalette(ROOT.kBird)
ROOT.ildStyle.SetNumberContours(256)

In [5]:
ROOT.gInterpreter.Declare("#include <WWTools.h>")

True

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
df = ROOT.RDataFrame("events", "data/stage1/test/sw_sl/enuqq_nano.root")
# df = ROOT.RDataFrame("events", "data/stage1/test/sw_sl/enuqq_simple_whizard_nano.root")
# df = ROOT.RDataFrame("events", "data/stage1/test/sw_sl/enuqq_simple_whizard_ISR_nano.root")
# df = ROOT.RDataFrame("events", "data/stage1/test/sw_sl/enuqq_simple_whizard_FCC_BES_ISR_nano.root")
# df = ROOT.RDataFrame("events", "data/stage1/test/sw_sl/enuqq_simple_madgraph_nano.root")
df.Describe()

Dataframe from TChain events in file data/stage1/test/sw_sl/enuqq_nano.root

Property                Value
--------                -----
Columns in total           42
Columns from defines        0
Event loops run             0
Processing slots            6

Column                          Type                                                            Origin
------                          ----                                                            ------
E_miss_lvec                     ROOT::Math::LorentzVector<ROOT::Math::PxPyPzE4D<double> >       Dataset
E_miss_lvec.fCoordinates        ROOT::Math::PxPyPzE4D<double>                                   Dataset
E_miss_lvec.fCoordinates.fT     Double_t                                                        Dataset
E_miss_lvec.fCoordinates.fX     Double_t                                                        Dataset
E_miss_lvec.fCoordinates.fY     Double_t                                                        Dataset
E_miss_lvec.fCoo

In [8]:
df = df.Vary("nu_lvec", "ROOT::Math::PxPyPzMVector tmp(E_miss_lvec); return RVec<ROOT::Math::PxPyPzMVector>{tmp};", ["recoable_nu"], "nu_variation")
# df = df.Vary(("nu_lvec", "l_lvec", "hadronic_lvec"), """
#               ROOT::Math::PxPyPzMVector tmp(E_miss_lvec);
#               auto boost = collision_lvec.BoostToCM();
#               return RVec<RVec<ROOT::Math::PxPyPzMVector>>{
#                 {ROOT::Math::VectorUtil::boost(nu_lvec, boost), ROOT::Math::VectorUtil::boost(tmp, boost)},
#                 {ROOT::Math::VectorUtil::boost(l_lvec, boost), ROOT::Math::VectorUtil::boost(l_lvec, boost)},
#                 {ROOT::Math::VectorUtil::boost(hadronic_lvec, boost), ROOT::Math::VectorUtil::boost(hadronic_lvec, boost)}
#               };
# """, ["boosted", "boosted_and_emiss"], "initial_lvecs")

In [9]:
# pdg 11 is e-, -11 is e+
df = df.Define("l_MC_charge", "-1 * ROOT::Math::Sign(l_pdg)")

### start of angle calculations
# the simplest is the production angle theta_W (where W stands not for Weinberg in this case but for W-boson)
# following M. Bilenky et al. in Nucl.Phys.B 409 (1993) 22-68 (doi: https://doi.org/10.1016/0550-3213(93)90445-U)
# this is the angle between the W^{-} and the beam e^{-} which in our case is just the Z-axis
# (TODO: check what happens when ddsim adds a crossing angle :/)

# check l charge to determine which ICN is W-
# TODO: normally the direction of the leptonically decaying W would need to be inferred from the hardronic one
df = df.Define("W_lep_lvec", "l_lvec + nu_lvec")
# weird addition order instead of subtraction because of types...
# wow but - is not correctly implemented for PxPyPzM vector...
df = df.Vary("W_lep_lvec", "RVec<ROOT::Math::PxPyPzMVector>{l_lvec + E_miss_lvec, -hadronic_lvec + ROOT::Math::PxPyPzEVector(0, 0, 0, 250)}",
            ["l_plus_miss", "sqrtS_minus_had"], "W_reco_strategy")

df = df.Define("W_had_lvec", "hadronic_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
# df = df.Define("e_plus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
# df = df.Define("lep_axis", "l_MC_charge > 0 ? e_plus_vec : e_minus_vec")
# df = df.Define("had_axis", "l_MC_charge < 0 ? e_plus_vec : e_minus_vec")
# df = df.Define("CosTheta_W_lep", "ROOT::Math::VectorUtil::CosTheta(W_lep_lvec, lep_axis)")
# df = df.Define("CosTheta_W_had", "ROOT::Math::VectorUtil::CosTheta(W_had_lvec, had_axis)")
df = df.Define("CosTheta_W_lep", "ROOT::Math::VectorUtil::CosTheta(W_lep_lvec, e_minus_vec)")
df = df.Define("CosTheta_W_had", "ROOT::Math::VectorUtil::CosTheta(W_had_lvec, e_minus_vec)")

# in LEP parametrisation this is _always_ e_minus_vec
# df = df.Define("l_star_lvec", "WWTools::starVector(W_lep_lvec, l_lvec, e_minus_vec)")
df = df.Define("l_star_lvec", "WWTools::starVector(W_lep_lvec, l_lvec)")

In [10]:
df = df.Define("l_theta_star", "l_star_lvec.Theta()")
df = df.Define("l_phi_star", "l_star_lvec.Phi()")
# shift by pi for positrons/antifermion
df = df.Define("l_phi_star_shift", "if (l_MC_charge < 0) return l_phi_star; auto phi = l_phi_star + ROOT::Math::Pi(); return phi > ROOT::Math::Pi() ? phi - 2 * ROOT::Math::Pi() : phi;")

df = df.Define("l_cosTheta_star", "cos(l_theta_star)")

df = df.Define("lnu_m", "W_lep_lvec.M()")
df = df.Define("had_m", "W_had_lvec.M()")
# FIXME hardcoded beam energy
# beam particle minus neutrino -> l charge -1, take beam positron, points in negative direction
df = df.Define("q2", "(ROOT::Math::PxPyPzEVector(0., 0., l_MC_charge * 125., 125.) - nu_lvec).M2()")

In [11]:
# FIXME: for debugging
# df = df.Redefine("weight", "1")
# df = df.Filter("l_MC_lvec_pt > 1", "pt cut")
# df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg
# df = df.Filter("beamPol_e != beamPol_p")
# df = df.Filter("beamPol_e < 0 && beamPol_p < 0") # LL
# df = df.Filter("beamPol_e < 0 && beamPol_p > 0") # LR
# df = df.Filter("beamPol_e > 0 && beamPol_p < 0") # RL
# df = df.Filter("beamPol_e > 0 && beamPol_p > 0") # RR
# df = df.Filter("abs(cos(l_lvec.Theta())) < cos(0.05)") # lumi cal inner
# df = df.Filter("abs(l_lvec.Eta()) < 2.5") # madgraph default cut
# df = df.Filter("abs(W_lep_lvec.M() - 80) < 5")

In [12]:
h_CosTheta_W_minus_had = df.Filter("l_MC_charge > 0").Histo1D(("", ";cos #theta_{W^{-}_{had}}", 40, -1., 1.), "CosTheta_W_had", "weight")
h_CosTheta_W_minus_lep = df.Filter("l_MC_charge < 0").Histo1D(("", ";cos #theta_{W^{-}_{lep}}", 40, -1., 1.), "CosTheta_W_lep", "weight")
h_CosTheta_W_plus_had = df.Filter("l_MC_charge < 0").Histo1D(("", ";cos #theta_{W^{+}_{had}}", 40, -1., 1.), "CosTheta_W_had", "weight")
h_CosTheta_W_plus_lep = df.Filter("l_MC_charge > 0").Histo1D(("", ";cos #theta_{W^{+}_{lep}}", 40, -1., 1.), "CosTheta_W_lep", "weight")

vh_CosTheta_W_minus_lep = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_minus_lep)
vh_CosTheta_W_plus_lep = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_plus_lep)

h_l_minus_cosTheta_star = df.Filter("l_MC_charge < 0").Histo1D(("", ";cos #theta*_{e^{-}}", 40, -1., 1.), "l_cosTheta_star", "weight")
h_l_plus_cosTheta_star = df.Filter("l_MC_charge > 0").Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "l_cosTheta_star", "weight")

vh_l_minus_cosTheta_star = ROOT.RDF.Experimental.VariationsFor(h_l_minus_cosTheta_star)
vh_l_plus_cosTheta_star = ROOT.RDF.Experimental.VariationsFor(h_l_plus_cosTheta_star)

h_l_minus_phi_star = df.Filter("l_MC_charge < 0").Histo1D(("", ";#phi*_{e-}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "l_phi_star", "weight")
h_l_plus_phi_star = df.Filter("l_MC_charge > 0").Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "l_phi_star", "weight")
h_l_plus_phi_star_shift = df.Filter("l_MC_charge > 0").Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "l_phi_star_shift", "weight")

vh_l_minus_phi_star = ROOT.RDF.Experimental.VariationsFor(h_l_minus_phi_star)
vh_l_plus_phi_star = ROOT.RDF.Experimental.VariationsFor(h_l_plus_phi_star)

h_2d_l_phi_cosTheta_star = df.Histo2D(("", ";#phi*_{e#pm};cos #theta*_{e^{#pm}}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi(), 40, -1., 1.), "l_phi_star_shift", "l_cosTheta_star", "weight")

h_lnu_m = df.Histo1D(("", ";M_{l+#nu} [GeV]", 350, 0., 350.), "lnu_m", "weight")
h_q2 = df.Histo1D(("", ";q^{2} [GeV^{2}]", 200, -60e3, 1e3), "q2", "weight")

In [13]:
h_l_minus_cosTheta_star.SetMinimum(0)
h_l_plus_cosTheta_star.SetMinimum(0)
h_l_minus_phi_star.SetMinimum(0)
h_l_plus_phi_star.SetMinimum(0)
h_l_plus_phi_star_shift.SetMinimum(0)

In [14]:
c_CosTheta_W_minus_had = ROOT.TCanvas()
h_CosTheta_W_minus_had.Draw()
c_CosTheta_W_minus_had.Draw()

c_CosTheta_W_minus_lep = ROOT.TCanvas()
h_CosTheta_W_minus_lep.Draw()
vh_CosTheta_W_minus_lep = ROOT.RDF.Experimental.VariationsFor(h_CosTheta_W_minus_lep)
vh_CosTheta_W_minus_lep["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_CosTheta_W_minus_lep["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_CosTheta_W_minus_lep["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_CosTheta_W_minus_lep["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_CosTheta_W_minus_lep.Draw()

# c_CosTheta_W_minus_lep_r = ROOT.TCanvas()
# r_CosTheta_W_minus_lep_r = ROOT.TRatioPlot(h_CosTheta_W_minus_lep.GetPtr(), vh_CosTheta_W_minus_lep["nu_variation:recoable_nu"])
# # r_CosTheta_W_minus_lep_r = ROOT.TRatioPlot(h_CosTheta_W_minus_lep.GetPtr(), vh_CosTheta_W_minus_lep["initial_lvecs:boosted"])
# # r_CosTheta_W_minus_lep_r = ROOT.TRatioPlot(h_CosTheta_W_minus_lep.GetPtr(), vh_CosTheta_W_minus_lep["initial_lvecs:boosted_and_emiss"])
# r_CosTheta_W_minus_lep_r.Draw()
# c_CosTheta_W_minus_lep_r.Draw()

c_CosTheta_W_plus_had = ROOT.TCanvas()
h_CosTheta_W_plus_had.Draw()
c_CosTheta_W_plus_had.Draw()

c_CosTheta_W_plus_lep = ROOT.TCanvas()
h_CosTheta_W_plus_lep.Draw()
c_CosTheta_W_plus_lep.Draw()

c_l_minus_cosTheta_star = ROOT.TCanvas()
h_l_minus_cosTheta_star.Draw()
vh_l_minus_cosTheta_star = ROOT.RDF.Experimental.VariationsFor(h_l_minus_cosTheta_star)
vh_l_minus_cosTheta_star["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_l_minus_cosTheta_star["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_l_minus_cosTheta_star["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_l_minus_cosTheta_star["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_l_minus_cosTheta_star.Draw()

c_l_plus_cosTheta_star = ROOT.TCanvas()
h_l_plus_cosTheta_star.Draw()
vh_l_plus_cosTheta_star = ROOT.RDF.Experimental.VariationsFor(h_l_plus_cosTheta_star)
vh_l_plus_cosTheta_star["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_l_plus_cosTheta_star["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_l_plus_cosTheta_star["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_l_plus_cosTheta_star["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_l_plus_cosTheta_star.Draw()

c_l_minus_phi_star = ROOT.TCanvas()
h_l_minus_phi_star.Draw()
vh_l_minus_phi_star = ROOT.RDF.Experimental.VariationsFor(h_l_minus_phi_star)
vh_l_minus_phi_star["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_l_minus_phi_star["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_l_minus_phi_star["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_l_minus_phi_star["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_l_minus_phi_star.Draw()

c_l_plus_phi_star = ROOT.TCanvas()
h_l_plus_phi_star.Draw()
vh_l_plus_phi_star = ROOT.RDF.Experimental.VariationsFor(h_l_plus_phi_star)
vh_l_plus_phi_star["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_l_plus_phi_star["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_l_plus_phi_star["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_l_plus_phi_star["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_l_plus_phi_star.Draw()

c_l_plus_phi_star_shift = ROOT.TCanvas()
h_l_plus_phi_star_shift.Draw()
vh_l_plus_phi_star_shift = ROOT.RDF.Experimental.VariationsFor(h_l_plus_phi_star_shift)
vh_l_plus_phi_star_shift["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_l_plus_phi_star_shift["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_l_plus_phi_star_shift["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_l_plus_phi_star_shift["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_l_plus_phi_star_shift.Draw()

c_2d_l_phi_cosTheta_star = ROOT.TCanvas()
h_2d_l_phi_cosTheta_star.Draw("colz")
c_2d_l_phi_cosTheta_star.Draw()

c_lnu_m = ROOT.TCanvas()
h_lnu_m.Draw()
vh_lnu_m = ROOT.RDF.Experimental.VariationsFor(h_lnu_m)
vh_lnu_m["W_reco_strategy:l_plus_miss"].SetMarkerColor(ROOT.kRed)
vh_lnu_m["W_reco_strategy:l_plus_miss"].Draw("same")
# vh_lnu_m["W_reco_strategy:sqrtS_minus_had"].SetMarkerColor(ROOT.kBlue)
# vh_lnu_m["W_reco_strategy:sqrtS_minus_had"].Draw("same")
c_lnu_m.Draw()

c_q2 = ROOT.TCanvas()
h_q2.Draw()
vh_q2 = ROOT.RDF.Experimental.VariationsFor(h_q2)
vh_q2["nu_variation:recoable_nu"].SetMarkerColor(ROOT.kRed)
vh_q2["nu_variation:recoable_nu"].Draw("same")
c_q2.Draw()

In [15]:
vh_q2.GetKeys()

vector<string>{ "nominal", "nu_variation:recoable_nu" }